In [1]:
import os
from datetime import datetime
from dateutil.parser import parse
import time

import numpy as np
import pandas as pd
import scipy.stats as scs
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from pprint import pprint
from tqdm import tqdm

import seaborn as sns
sns.set_theme(style="whitegrid")

from src.nft_analytics import NFTAnalytics
from src.ethereum_api import EthereumAPI

In [2]:
gaa = NFTAnalytics("0xA0F38233688bB578c0a88102A95b846c18bc0bA7")
DATA_FOLDER, RESULT_FOLDER = gaa.make_directories("GreatApeSociety")
TIMESTAMP = datetime.now().strftime("%Y-%m-%d_")

2021-10-15 14:06:53,137 - src.ethereum_api - INFO - Connected to Infura endpoint.


In [3]:
asset_data = gaa.load_json(filename=os.path.join(DATA_FOLDER, "data.json"))

In [7]:
def get_rarities(asset_data: list) -> dict:
    rarity_scores = []
    token_ids = []

    for asset in asset_data:        
        rarity = 0
        if asset["traits"]:    
            for trait in asset["traits"]:
                rarity += 1 / int(trait["trait_count"])
            token_ids.append(asset["token_id"])
            rarity_scores.append(rarity)

    rarity_scores = np.array(rarity_scores)
    rarity_scores = (100 - 1) / (max(rarity_scores) - min(rarity_scores)) * (rarity_scores - max(rarity_scores)) + 100

    rarities = {}
    for token_id, rarity_score in zip(token_ids, rarity_scores):
        rarities[token_id] = rarity_score
        
    return rarities

In [8]:
%%timeit
r = get_rarities(asset_data)

34.6 ms ± 1.91 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
